In [ ]:
# Install required packages for hybrid retrieval (sentence transformer + BM25)
%pip install sentence-transformers torch datasets scikit-learn pandas numpy tqdm faiss-cpu
%pip install nltk transformers  # For tokenization and processing
%pip install rank-bm25  # For BM25 sparse retrieval integration


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import os
# Disable wandb logging FIRST - before any other imports
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"

import pandas as pd
import numpy as np
import torch
import re
import random
from collections import defaultdict
import xml.etree.ElementTree as ET
from tqdm import tqdm
import warnings
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
warnings.filterwarnings('ignore')

from sentence_transformers import SentenceTransformer, InputExample, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import faiss

# BM25 and hybrid retrieval imports
from rank_bm25 import BM25Okapi
from typing import List, Dict, Tuple, Union

# Download NLTK data
try:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
except:
    pass

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


Using device: cuda


In [ ]:
def calculate_semantic_diversity(text1, text2):
    """Calculate semantic diversity - OPPOSITE of keyword overlap!

    This function encourages semantically DIFFERENT negatives,
    not lexically similar ones that confuse the model.
    """
    # Simple diversity metric based on character-level differences
    chars1 = set(text1.lower())
    chars2 = set(text2.lower())

    # Calculate character diversity (higher = more diverse = better negative)
    intersection = len(chars1 & chars2)
    union = len(chars1 | chars2)

    if union == 0:
        return 1.0

    # Return DIVERSITY score (1 - similarity) - opposite of overlap!
    diversity = 1.0 - (intersection / union)
    return diversity

def semantic_negative_sampling(all_doc_ids, excluded_doc_ids, documents, query_text, num_samples=100):
    """Sample negatives with preference for semantic diversity - NO lexical bias!"""
    available_docs = [doc_id for doc_id in all_doc_ids if doc_id not in excluded_doc_ids]

    if len(available_docs) == 0:
        return []

    # Sample diverse candidates
    sample_size = min(num_samples, len(available_docs))
    sampled_docs = np.random.choice(available_docs, size=sample_size, replace=False)

    # Calculate DIVERSITY scores (not similarity!)
    diversity_scores = []
    for doc_id in sampled_docs:
        if doc_id in documents:
            diversity = calculate_semantic_diversity(query_text, documents[doc_id])
            diversity_scores.append((doc_id, diversity))

    # Sort by DIVERSITY (descending) - prefer DIFFERENT content
    diversity_scores.sort(key=lambda x: x[1], reverse=True)

    return [doc_id for doc_id, _ in diversity_scores]

def create_semantic_training_triplets(queries, documents, qrels, num_negatives_per_positive=4):
    """Create training triplets for sentence transformer with SEMANTIC focus - NO keyword overlap bias!"""
    training_examples = []
    all_doc_ids = list(documents.keys())

    print("🧠 Creating SEMANTIC training triplets for Sentence Transformer...")
    print("❌ REMOVED: Keyword overlap-based negative mining")
    print("✅ ADDED: Semantic diversity-based negative sampling")
    print("✅ OPTIMIZED: For sentence transformer architecture")

    for query_id, query_text in tqdm(queries.items(), desc="Semantic triplet creation"):
        if query_id not in qrels:
            continue

        relevant_docs = [doc_id for doc_id, rel in qrels[query_id].items() if rel == 1]
        non_relevant_docs = [doc_id for doc_id, rel in qrels[query_id].items() if rel == 0]

        if not relevant_docs:
            continue

        for pos_doc_id in relevant_docs:
            if pos_doc_id not in documents:
                continue

            pos_doc_text = documents[pos_doc_id]

            # Strategy 1: Use labeled non-relevant documents (50%)
            labeled_negatives = [neg_id for neg_id in non_relevant_docs if neg_id in documents]

            # Strategy 2: Sample semantically diverse random negatives (50%)
            used_doc_ids = set(qrels[query_id].keys())
            diverse_negatives = semantic_negative_sampling(
                all_doc_ids, used_doc_ids, documents, query_text, num_samples=50
            )

            # Combine and balance negative sources
            all_negatives = labeled_negatives + diverse_negatives[:num_negatives_per_positive]

            # Randomly sample final negatives for balanced training
            if len(all_negatives) > num_negatives_per_positive:
                selected_negatives = np.random.choice(
                    all_negatives, size=num_negatives_per_positive, replace=False
                )
            else:
                selected_negatives = all_negatives

            # Create InputExamples for sentence transformer triplet training
            for neg_doc_id in selected_negatives:
                if neg_doc_id in documents:
                    # Create triplet: anchor=query, positive=relevant_doc, negative=irrelevant_doc
                    example = InputExample(
                        texts=[query_text, pos_doc_text, documents[neg_doc_id]],
                        label=0.0  # For triplet loss, the negative should have lower similarity
                    )
                    training_examples.append(example)

    return training_examples


In [ ]:
class SemanticSentenceTransformer:
    """Enhanced Sentence Transformer with semantic understanding for code-mixed Bengali IR"""

    def __init__(self, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', max_seq_length=512):
        """
        Initialize with Hugging Face multilingual sentence transformer

        Args:
            model_name: Hugging Face model name
                       'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2' - Best for multilingual
                       'sentence-transformers/all-MiniLM-L6-v2' - Faster, English-focused
                       'sentence-transformers/paraphrase-multilingual-mpnet-base-v2' - Higher quality
        """
        self.model_name = model_name
        self.max_seq_length = max_seq_length

        print(f"🚀 Initializing Semantic Sentence Transformer from Hugging Face: {model_name}")
        print("✅ Pre-optimized for semantic similarity")
        print("✅ Multilingual support for Bengali/Roman")
        print("✅ Efficient architecture for faster training")
        print("🚫 Wandb logging disabled")

        # Load pre-trained sentence transformer from Hugging Face
        self.model = SentenceTransformer(model_name)

        # Set maximum sequence length
        self.model.max_seq_length = max_seq_length

        # Move to device
        self.model = self.model.to(device)

        print(f"📊 Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        print(f"📏 Max sequence length: {max_seq_length}")

    def fine_tune_semantic(self, training_examples, val_examples=None,
                          epochs=2, batch_size=8, warmup_steps=50,
                          output_path='best_sentence_transformer_semantic_ir'):
        """Fine-tune with semantic-aware triplet loss - NO WANDB"""

        print(f"\n🧠 Fine-tuning with {len(training_examples)} semantic triplets...")
        print("🎯 Focus: Semantic understanding over lexical matching")
        print("⚡ Advantage: Pre-trained semantic representations")
        print("🚫 Training without wandb logging")

        # Create data loader
        train_dataloader = DataLoader(training_examples, shuffle=True, batch_size=batch_size)

        # Use TripletLoss - optimized for semantic ranking
        train_loss = losses.TripletLoss(model=self.model)

        # Training arguments
        total_steps = len(train_dataloader) * epochs

        print(f"\n📈 Training configuration:")
        print(f"  Epochs: {epochs}")
        print(f"  Batch size: {batch_size}")
        print(f"  Total steps: {total_steps}")
        print(f"  Warmup steps: {warmup_steps}")
        print(f"  Loss function: Triplet Loss (semantic ranking)")
        print(f"  Logging: Disabled (no wandb required)")

        # Evaluator (optional)
        evaluator = None
        if val_examples and len(val_examples) > 0:
            # Create evaluation examples from validation set
            eval_examples = []
            for example in val_examples[:50]:  # Smaller sample for efficiency
                # For evaluation: query-positive should have high similarity
                eval_examples.append(InputExample(texts=[example.texts[0], example.texts[1]], label=1.0))
                # query-negative should have low similarity
                eval_examples.append(InputExample(texts=[example.texts[0], example.texts[2]], label=0.0))

            evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
                eval_examples, name='semantic_ir_eval'
            )
            print(f"  Evaluation: {len(eval_examples)} examples for validation")

        # Fine-tune the model with explicit wandb disable
        print("\n🔄 Starting training...")
        self.model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            epochs=epochs,
            warmup_steps=warmup_steps,
            output_path=output_path,
            evaluator=evaluator,
            evaluation_steps=1000,  # Less frequent evaluation
            save_best_model=True,
            show_progress_bar=True,
            use_amp=False  # Disable automatic mixed precision for stability
        )

        print(f"\n✅ Fine-tuning completed!")
        print(f"📁 Model saved to: {output_path}")

        # Load the best model
        self.model = SentenceTransformer(output_path)

        return output_path

    def encode(self, texts, batch_size=32, show_progress_bar=True, convert_to_tensor=False):
        """Encode texts into semantic embeddings"""
        return self.model.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress_bar,
            convert_to_tensor=convert_to_tensor,
            device=device
        )

    def similarity(self, embeddings1, embeddings2):
        """Calculate semantic similarity between embeddings"""
        return util.cos_sim(embeddings1, embeddings2)

    def save(self, path):
        """Save the model"""
        self.model.save(path)

    def load(self, path):
        """Load a saved model"""
        self.model = SentenceTransformer(path).to(device)


In [ ]:
class HybridRetriever:
    """
    Hybrid Retrieval System combining fine-tuned Sentence Transformer (dense) with BM25 (sparse)

    This class implements state-of-the-art hybrid retrieval that combines:
    - Dense retrieval: Semantic similarity via fine-tuned sentence transformers
    - Sparse retrieval: Lexical matching via BM25
    - Score fusion: Multiple fusion strategies for optimal results
    """

    def __init__(self, semantic_model, documents: Dict[int, str],
                 bm25_k1: float = 1.2, bm25_b: float = 0.75):
        """
        Initialize hybrid retriever

        Args:
            semantic_model: Fine-tuned SentenceTransformer model
            documents: Dictionary of {doc_id: document_text}
            bm25_k1: BM25 parameter k1 (term frequency saturation)
            bm25_b: BM25 parameter b (document length normalization)
        """
        self.semantic_model = semantic_model
        self.documents = documents
        self.doc_ids = list(documents.keys())
        self.doc_texts = [documents[doc_id] for doc_id in self.doc_ids]

        print(f"🔧 Initializing Hybrid Retriever with {len(documents)} documents...")
        print(f"📊 BM25 parameters: k1={bm25_k1}, b={bm25_b}")

        # Initialize BM25
        print("🔍 Building BM25 index...")
        self._build_bm25_index(bm25_k1, bm25_b)

        # Initialize dense retrieval
        print("🧠 Building semantic embeddings index...")
        self._build_semantic_index()

        print("✅ Hybrid retriever initialized successfully!")

    def _preprocess_for_bm25(self, text: str) -> List[str]:
        """Preprocess text for BM25 - tokenization and basic cleaning"""
        # Basic tokenization - important for code-mixed Bengali
        text = re.sub(r'[^\w\s]', ' ', text.lower())
        tokens = text.split()

        # Remove very short tokens and numbers
        tokens = [token for token in tokens if len(token) > 1 and not token.isdigit()]

        return tokens

    def _build_bm25_index(self, k1: float, b: float):
        """Build BM25 index for sparse retrieval"""
        # Tokenize all documents
        tokenized_docs = [self._preprocess_for_bm25(doc) for doc in self.doc_texts]

        # Create BM25 index
        self.bm25 = BM25Okapi(tokenized_docs, k1=k1, b=b)

        print(f"   ✅ BM25 index built with {len(tokenized_docs)} documents")

    def _build_semantic_index(self):
        """Build semantic embeddings index for dense retrieval"""
        # Encode all documents
        self.doc_embeddings = self.semantic_model.encode(
            self.doc_texts,
            batch_size=32,
            show_progress_bar=True,
            convert_to_tensor=False
        )

        # Create FAISS index
        dimension = self.doc_embeddings.shape[1]
        self.semantic_index = faiss.IndexFlatIP(dimension)
        faiss.normalize_L2(self.doc_embeddings)
        self.semantic_index.add(self.doc_embeddings)

        print(f"   ✅ Semantic index built with {self.semantic_index.ntotal} documents")

    def bm25_search(self, query: str, top_k: int = 1000) -> List[Tuple[int, float]]:
        """Perform BM25 sparse retrieval"""
        query_tokens = self._preprocess_for_bm25(query)
        scores = self.bm25.get_scores(query_tokens)

        # Get top-k results
        doc_scores = [(self.doc_ids[i], scores[i]) for i in range(len(scores))]
        doc_scores.sort(key=lambda x: x[1], reverse=True)

        return doc_scores[:top_k]

    def semantic_search(self, query: str, top_k: int = 1000) -> List[Tuple[int, float]]:
        """Perform semantic dense retrieval"""
        query_embedding = self.semantic_model.encode([query], convert_to_tensor=False)
        faiss.normalize_L2(query_embedding)

        scores, indices = self.semantic_index.search(query_embedding, top_k)

        doc_scores = []
        for score, idx in zip(scores[0], indices[0]):
            if idx != -1:
                doc_scores.append((self.doc_ids[idx], float(score)))

        return doc_scores

    def normalize_scores(self, scores: List[float]) -> List[float]:
        """Normalize scores to [0, 1] range using min-max normalization"""
        if not scores:
            return scores

        min_score = min(scores)
        max_score = max(scores)

        if max_score == min_score:
            return [1.0] * len(scores)

        return [(score - min_score) / (max_score - min_score) for score in scores]

    def weighted_fusion(self, semantic_results: List[Tuple[int, float]],
                       bm25_results: List[Tuple[int, float]],
                       semantic_weight: float = 0.7,
                       top_k: int = 1000) -> List[Tuple[int, float]]:
        """
        Combine results using weighted score fusion

        Args:
            semantic_results: Results from semantic search
            bm25_results: Results from BM25 search
            semantic_weight: Weight for semantic scores (0-1)
            top_k: Number of results to return
        """
        bm25_weight = 1.0 - semantic_weight

        # Convert to dictionaries for easier lookup
        semantic_dict = dict(semantic_results)
        bm25_dict = dict(bm25_results)

        # Normalize scores separately
        if semantic_results:
            semantic_scores = [score for _, score in semantic_results]
            norm_semantic_scores = self.normalize_scores(semantic_scores)
            semantic_dict = {doc_id: norm_score for (doc_id, _), norm_score in zip(semantic_results, norm_semantic_scores)}

        if bm25_results:
            bm25_scores = [score for _, score in bm25_results]
            norm_bm25_scores = self.normalize_scores(bm25_scores)
            bm25_dict = {doc_id: norm_score for (doc_id, _), norm_score in zip(bm25_results, norm_bm25_scores)}

        # Combine scores
        all_doc_ids = set(semantic_dict.keys()) | set(bm25_dict.keys())
        combined_results = []

        for doc_id in all_doc_ids:
            semantic_score = semantic_dict.get(doc_id, 0.0)
            bm25_score = bm25_dict.get(doc_id, 0.0)

            combined_score = (semantic_weight * semantic_score +
                            bm25_weight * bm25_score)
            combined_results.append((doc_id, combined_score))

        # Sort by combined score
        combined_results.sort(key=lambda x: x[1], reverse=True)

        return combined_results[:top_k]

    def reciprocal_rank_fusion(self, semantic_results: List[Tuple[int, float]],
                              bm25_results: List[Tuple[int, float]],
                              k: int = 60, top_k: int = 1000) -> List[Tuple[int, float]]:
        """
        Combine results using Reciprocal Rank Fusion (RRF)

        RRF is often more robust than weighted fusion as it doesn't require score normalization

        Args:
            semantic_results: Results from semantic search
            bm25_results: Results from BM25 search
            k: RRF parameter (typically 60)
            top_k: Number of results to return
        """
        # Create rank dictionaries
        semantic_ranks = {doc_id: rank for rank, (doc_id, _) in enumerate(semantic_results)}
        bm25_ranks = {doc_id: rank for rank, (doc_id, _) in enumerate(bm25_results)}

        # Calculate RRF scores
        all_doc_ids = set(semantic_ranks.keys()) | set(bm25_ranks.keys())
        rrf_results = []

        for doc_id in all_doc_ids:
            rrf_score = 0.0

            if doc_id in semantic_ranks:
                rrf_score += 1.0 / (k + semantic_ranks[doc_id] + 1)

            if doc_id in bm25_ranks:
                rrf_score += 1.0 / (k + bm25_ranks[doc_id] + 1)

            rrf_results.append((doc_id, rrf_score))

        # Sort by RRF score
        rrf_results.sort(key=lambda x: x[1], reverse=True)

        return rrf_results[:top_k]

    def hybrid_search(self, query: str, method: str = 'weighted',
                     semantic_weight: float = 0.7, rrf_k: int = 60,
                     top_k: int = 1000) -> List[Tuple[int, float]]:
        """
        Perform hybrid search combining semantic and BM25 retrieval

        Args:
            query: Search query
            method: Fusion method ('weighted' or 'rrf')
            semantic_weight: Weight for semantic scores (only for weighted method)
            rrf_k: RRF parameter (only for rrf method)
            top_k: Number of results to return
        """
        # Get results from both methods
        semantic_results = self.semantic_search(query, top_k)
        bm25_results = self.bm25_search(query, top_k)

        # Combine using specified method
        if method == 'weighted':
            return self.weighted_fusion(semantic_results, bm25_results,
                                      semantic_weight, top_k)
        elif method == 'rrf':
            return self.reciprocal_rank_fusion(semantic_results, bm25_results,
                                             rrf_k, top_k)
        else:
            raise ValueError(f"Unknown fusion method: {method}")

    def analyze_query_coverage(self, query: str, top_k: int = 100) -> Dict:
        """Analyze how semantic and BM25 methods complement each other for a query"""
        semantic_results = self.semantic_search(query, top_k)
        bm25_results = self.bm25_search(query, top_k)

        semantic_docs = set(doc_id for doc_id, _ in semantic_results)
        bm25_docs = set(doc_id for doc_id, _ in bm25_results)

        overlap = semantic_docs & bm25_docs
        semantic_only = semantic_docs - bm25_docs
        bm25_only = bm25_docs - semantic_docs

        return {
            'overlap_count': len(overlap),
            'semantic_only_count': len(semantic_only),
            'bm25_only_count': len(bm25_only),
            'overlap_percentage': len(overlap) / top_k * 100,
            'semantic_unique_percentage': len(semantic_only) / top_k * 100,
            'bm25_unique_percentage': len(bm25_only) / top_k * 100,
            'total_unique_docs': len(semantic_docs | bm25_docs)
        }


In [ ]:
# Bengali to Banglish Converter for Test Query Alignment

import re
from typing import Dict, List
import pandas as pd

def load_trec_queries(file_path):
    """Parse TREC format query file"""
    queries = []

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Parse XML-like structure
    topics = re.findall(r'<top>(.*?)</top>', content, re.DOTALL)

    for topic in topics:
        # Extract query number
        num_match = re.search(r'<num>(.*?)</num>', topic, re.DOTALL)
        if not num_match:
            continue
        query_id = int(num_match.group(1).strip())

        # Extract title
        title_match = re.search(r'<title>(.*?)</title>', topic, re.DOTALL)
        if title_match:
            title = title_match.group(1).strip()
        else:
            title = ""

        # Extract description (optional)
        desc_match = re.search(r'<desc>(.*?)</desc>', topic, re.DOTALL)
        desc = desc_match.group(1).strip() if desc_match else ""

        # Combine title and description
        query_text = f"{title} {desc}".strip()

        if query_text:
            queries.append({"qid":str(query_id),"query":query_text.strip() })
            # queries[query_id] = query_text

    return queries
# def load_trec_queries(filepath: str) -> List[Dict]:
#     queries = []
#     with open(filepath, "r", encoding="utf-8") as f:
#         content = f.read()
#     pattern = r"<num>(\d+)</num>.?<title>(.?)</title>"
#     matches = re.findall(pattern, content, re.DOTALL)
#     print(f"hello: {matches}")
#     for qid, title in matches:
#         queries.append({"qid": qid.strip(), "query": title.strip()})
#     return queries


class BengaliToBanglishConverter:
    """Convert Bengali Unicode text to Banglish (romanized) for model compatibility"""

    def __init__(self):
        # Enhanced Bengali to Banglish mapping
        self.bengali_to_banglish = {
            # Vowels
            "আ": "a",
            "ই": "i",
            "ঈ": "i",
            "উ": "u",
            "ঊ": "u",
            "ঋ": "ri",
            "এ": "e",
            "ঐ": "oi",
            "ও": "o",
            "ঔ": "ou",
            # Consonants with inherent 'a'
            "ক": "ka",
            "খ": "kha",
            "গ": "ga",
            "ঘ": "gha",
            "ঙ": "nga",
            "চ": "cha",
            "ছ": "chha",
            "জ": "ja",
            "ঝ": "jha",
            "ঞ": "nja",
            "ট": "ta",
            "ঠ": "tha",
            "ড": "da",
            "ঢ": "dha",
            "ণ": "na",
            "ত": "ta",
            "থ": "tha",
            "দ": "da",
            "ধ": "dha",
            "ন": "na",
            "প": "pa",
            "ফ": "pha",
            "ব": "ba",
            "ভ": "bha",
            "ম": "ma",
            "য": "ja",
            "র": "ra",
            "ল": "la",
            "শ": "sha",
            "ষ": "sha",
            "স": "sa",
            "হ": "ha",
            "ড়": "ra",
            "ঢ়": "rha",
            "য়": "ya",
            "ৎ": "t",
            "ং": "ng",
            "ঃ": "h",
            "ঁ": "n",
            # Vowel diacritics (কার)
            "া": "a",
            "ি": "i",
            "ী": "i",
            "ু": "u",
            "ূ": "u",
            "ৃ": "ri",
            "ে": "e",
            "ৈ": "oi",
            "ো": "o",
            "ৌ": "ou",
            # Numbers
            "০": "0",
            "১": "1",
            "২": "2",
            "৩": "3",
            "৪": "4",
            "৫": "5",
            "৬": "6",
            "৭": "7",
            "৮": "8",
            "৯": "9",
            # Special symbols
            "।": ".",
            "॥": ".",
        }

        # Common Bengali words to Banglish (most frequent)
        self.word_mappings = {
            "আমি": "ami",
            "তুমি": "tumi",
            "তোমার": "tomar",
            "আমার": "amar",
            "কি": "ki",
            "কেমন": "kemon",
            "কোথায়": "kothay",
            "কবে": "kobe",
            "কেন": "keno",
            "কিভাবে": "kivabe",
            "ভালো": "valo",
            "ভাল": "valo",
            "খারাপ": "kharap",
            "সুন্দর": "sundor",
            "বেশি": "beshi",
            "কম": "kom",
            "অনেক": "onek",
            "একটু": "ektu",
            "বড়": "boro",
            "ছোট": "choto",
            "ভাই": "vai",
            "বোন": "bon",
            "আপু": "apu",
            "মামা": "mama",
            "চাচা": "chacha",
            "খালা": "khala",
            "দাদা": "dada",
            "হবে": "hobe",
            "আছে": "ache",
            "নেই": "nei",
            "চাই": "chai",
            "দেখো": "dekho",
            "বলো": "bolo",
            "যাও": "jao",
            "এসো": "esho",
            "খাও": "khao",
            "পারো": "paro",
            "করো": "koro",
            "দাও": "dao",
            "নাও": "nao",
            "থাকো": "thako",
            "জানো": "jano",
            "বুঝো": "bujho",
            "এই": "ei",
            "সেই": "sei",
            "ওই": "oi",
            "এটা": "eta",
            "সেটা": "seta",
            "ওটা": "ota",
            "কোন": "kon",
            "কোনো": "kono",
            "সব": "sob",
            "আর": "ar",
            "ও": "o",
            "না": "na",
            "হ্যাঁ": "hya",
            "হ্যা": "hya",
            "একটা": "ekta",
            "দুটো": "duto",
            "তিনটে": "tinte",
            "চারটে": "charte",
            "পাঁচটা": "pachta",
            "ছয়টা": "choyeta",
            "সাতটা": "satta",
            "আটটা": "atta",
            "নয়টা": "noyeta",
            "দশটা": "doshta",
            "করছি": "korchi",
            "করছো": "korcho",
            "করছে": "korche",
            "করেছি": "korechi",
            "করেছো": "korecho",
            "করেছে": "koreche",
            "করব": "korbo",
            "করবে": "korbe",
            "করবো": "korbo",
            "যাচ্ছি": "jacchi",
            "যাচ্ছো": "jaccho",
            "যাচ্ছে": "jacche",
            "এসেছি": "eshechi",
            "এসেছো": "eshecho",
            "এসেছে": "esheche",
            "ভালোবাসি": "valobashi",
            "ভালোবাসা": "valobasha",
            "খুশি": "khushi",
            "দুঃখিত": "dukkhito",
            "রাগ": "rag",
            "বন্ধু": "bondhu",
            "পরিবার": "poribar",
            "বাড়ি": "bari",
            "স্কুল": "school",
            "কলেজ": "college",
            "অফিস": "office",
            "কাজ": "kaj",
            "পড়া": "pora",
            "লেখা": "lekha",
            "গান": "gan",
            "খেলা": "khela",
            "সিনেমা": "cinema",
            "বই": "boi",
            "ফোন": "phone",
            "মোবাইল": "mobile",
            "কম্পিউটার": "computer",
            "ইন্টারনেট": "internet",
            "ফেসবুক": "facebook",
        }

    def convert_word(self, bengali_word: str) -> str:
        """Convert a single Bengali word to Banglish"""
        if not bengali_word.strip():
            return bengali_word

        # Check direct word mapping first
        if bengali_word in self.word_mappings:
            return self.word_mappings[bengali_word]

        # Character-by-character conversion for unknown words
        result = ""
        i = 0
        word = bengali_word.strip()

        while i < len(word):
            char = word[i]

            # Handle conjuncts (যুক্তাক্ষর) - simplified approach
            if i < len(word) - 1:
                two_char = word[i : i + 2]
                if two_char in self.bengali_to_banglish:
                    result += self.bengali_to_banglish[two_char]
                    i += 2
                    continue

            # Single character conversion
            if char in self.bengali_to_banglish:
                result += self.bengali_to_banglish[char]
            elif char == "্":  # Hasanta (virama) - skip or handle specially
                if i < len(word) - 1:  # If not at end, might be conjunct
                    pass  # Skip hasanta, next consonant will be processed
                else:
                    result += char  # Keep if at end
            elif "\u0980" <= char <= "\u09ff":  # Bengali unicode range
                # Unknown Bengali character, try phonetic approximation
                result += "X"  # Placeholder
            else:
                # Non-Bengali character (English, numbers, punctuation)
                result += char

            i += 1

        return result

    def convert_text(self, bengali_text: str) -> str:
        """Convert full Bengali text to Banglish"""
        if not bengali_text:
            return ""

        # Clean and tokenize
        text = re.sub(r"\s+", " ", bengali_text.strip())
        words = text.split()

        # Convert each word
        banglish_words = []
        for word in words:
            # Remove punctuation for conversion, then add back
            clean_word = re.sub(r"[^\u0980-\u09FF\w]", "", word)
            punct = re.sub(r"[\u0980-\u09FF\w]", "", word)

            if clean_word:
                converted = self.convert_word(clean_word)
                banglish_words.append(converted + punct)
            elif punct:
                banglish_words.append(punct)

        return " ".join(banglish_words).lower()

    def convert_query_dataset(self, queries: List[Dict]) -> List[Dict]:
        """Convert a list of query dictionaries"""
        converted_queries = []

        print(f"🔄 Converting {len(queries)} Bengali queries to Banglish...")

        for query in queries:
            original_query = query["query"]
            converted_query = self.convert_text(original_query)

            converted_queries.append(
                {
                    "qid": query["qid"],
                    "query": converted_query,
                    "original_query": original_query,  # Keep for reference
                }
            )

            # Show conversion examples
            if len(converted_queries) <= 5:
                print(f"  {query['qid']}: '{original_query}' → '{converted_query}'")

        return converted_queries


# Updated data loading function for your main code
def load_and_convert_test_queries(
    filepath: str, converter: BengaliToBanglishConverter
) -> List[Dict]:
    """Load test queries and convert Bengali to Banglish"""

    # Load original Bengali queries
    bengali_queries = load_trec_queries(filepath)  # Your existing function

    # Convert to Banglish
    banglish_queries = converter.convert_query_dataset(bengali_queries)

    print(f"✅ Converted {len(banglish_queries)} test queries from Bengali to Banglish")

    # Save converted queries for inspection
    converted_df = pd.DataFrame(banglish_queries)
    converted_df.to_csv(
        "converted_test_queries_banglish.csv", index=False, encoding="utf-8"
    )
    print("💾 Saved converted queries to 'converted_test_queries_banglish.csv'")

    return banglish_queries


# Usage in your main code - REPLACE your test query loading:

# OLD
test_queries = load_trec_queries("data/test_query_30 (1).trec")
print(f"final queries: {test_queries}")

# NEW
converter = BengaliToBanglishConverter()
test_queries = load_and_convert_test_queries("data/test_query_30 (1).trec", converter)

# The rest of your code remains the same!
# Your normalizer will now work properly since queries are in Banglish

final_queries = {q["qid"]:q["query"] for q in test_queries}
print(f"final queries: {test_queries}")



final queries: [{'qid': '26', 'query': 'hi hyderabad e rapid antigen test kothay kora hochhe keu janate parben its urgent jate test korar 1 2 ghontar modhhe result pete pari hi hyderabad e rapid antigen test kothay kora hochhe keu janate parben its urgent jate test korar 1 2 ghontar modhhe result pete pari'}, {'qid': '27', 'query': 'hello everyone i need urgent information regarding travelling to west bengal this group has been very helpful to me so hope now also i shall get someone s help who has travelled to kolkata very recently i have to travel to kolkata airport and amar bari nadia district e so what is the procedure to collect the e pass for the interdistrict kolkata to nadia travel and do i need any document for interstate telangana to wb travel a hired car will come to pick me from the airport so what documents do i need to collect please let me know the details hello everyone i need urgent information regarding travelling to west bengal this group has been very helpful to me s

In [ ]:
def parse_trec_corpus(file_path):
    """Parse TREC format corpus file"""
    documents = {}

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split by document boundaries
    docs = re.split(r'<DOC>|</DOC>', content)

    for doc in docs:
        if not doc.strip():
            continue

        # Extract DOCNO
        docno_match = re.search(r'<DOCNO>(.*?)</DOCNO>', doc, re.DOTALL)
        if not docno_match:
            continue
        docno = docno_match.group(1).strip()

        # Extract BODY
        body_match = re.search(r'<BODY>(.*?)</BODY>', doc, re.DOTALL)
        if body_match:
            body = body_match.group(1).strip()
        else:
            body = ""

        # Extract HEAD (optional)
        head_match = re.search(r'<HEAD>(.*?)</HEAD>', doc, re.DOTALL)
        head = head_match.group(1).strip() if head_match else ""

        # Combine head and body
        full_text = f"{head} {body}".strip()

        if full_text:
            documents[int(docno)] = full_text

    return documents

def parse_trec_queries(file_path):
    """Parse TREC format query file"""
    queries = {}

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Parse XML-like structure
    topics = re.findall(r'<top>(.*?)</top>', content, re.DOTALL)

    for topic in topics:
        # Extract query number
        num_match = re.search(r'<num>(.*?)</num>', topic, re.DOTALL)
        if not num_match:
            continue
        query_id = int(num_match.group(1).strip())

        # Extract title
        title_match = re.search(r'<title>(.*?)</title>', topic, re.DOTALL)
        if title_match:
            title = title_match.group(1).strip()
        else:
            title = ""

        # Extract description (optional)
        desc_match = re.search(r'<desc>(.*?)</desc>', topic, re.DOTALL)
        desc = desc_match.group(1).strip() if desc_match else ""

        # Combine title and description
        query_text = f"{title} {desc}".strip()

        if query_text:
            queries[query_id] = query_text

    return queries

def parse_qrels(file_path):
    """Parse QRels file (query_id Q0 doc_id relevance)"""
    qrels = defaultdict(dict)

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 4:
                query_id = int(parts[0])
                doc_id = int(parts[2])
                relevance = int(parts[3])
                qrels[query_id][doc_id] = relevance

    return qrels

# Load all data
print("Loading corpus...")
documents = parse_trec_corpus('data/Baseline_Corpus.trec')
print(f"Loaded {len(documents)} documents")

print("\nLoading training queries...")
train_queries = parse_trec_queries('data/Train_query_20_Roman.trec')
print(f"Loaded {len(train_queries)} training queries")

print("\nLoading test queries...")
test_queries_real = parse_trec_queries('data/test_query_30 (1).trec')
test_queries = final_queries
print(f"Loaded {len(test_queries)} test queries")

print("\nLoading QRels...")
qrels = parse_qrels('data/QRels_Train_20.txt')
print(f"Loaded relevance judgments for {len(qrels)} queries")

# Display sample Bengali/Roman code-mixed data
print("\n=== Sample Bengali/Roman Code-Mixed Data ===")
sample_query_id = list(train_queries.keys())[0]
print(f"Sample Training Query {sample_query_id}: {train_queries[sample_query_id]}")

sample_test_query_id = list(test_queries.keys())[0]
print(f"Sample Test Query {sample_test_query_id}: {test_queries[sample_test_query_id]}")

print(f"\nSample QRel for Query {sample_query_id}: {dict(list(qrels[sample_query_id].items())[:5])}")


Loading corpus...
Loaded 107900 documents

Loading training queries...
Loaded 20 training queries

Loading test queries...
Loaded 30 test queries

Loading QRels...
Loaded relevance judgments for 20 queries

=== Sample Bengali/Roman Code-Mixed Data ===
Sample Training Query 1: hyderabad to howrah kono train ki diyeche ba debe durgapur jete hobe any idea jodi train chare then timing gulo ektu help korben
Sample Test Query 26: hi hyderabad e rapid antigen test kothay kora hochhe keu janate parben its urgent jate test korar 1 2 ghontar modhhe result pete pari hi hyderabad e rapid antigen test kothay kora hochhe keu janate parben its urgent jate test korar 1 2 ghontar modhhe result pete pari

Sample QRel for Query 1: {4: 0, 11: 0, 339: 0, 1378: 0, 1861: 1}


In [ ]:
# Create semantic-aware training triplets (NO keyword overlap bias!)
print("🧠 Creating semantic-aware training triplets for Sentence Transformer...")
print("🚫 REMOVED: Keyword overlap-based negative mining")
print("✅ ADDED: Semantic diversity-based negative sampling")
print("⚡ OPTIMIZED: For sentence transformer efficiency")

training_examples = create_semantic_training_triplets(
    train_queries, documents, qrels,
    num_negatives_per_positive=3  # Fewer negatives for faster training
)

print(f"\n📊 Created {len(training_examples)} semantic training triplets")

# Split for validation
train_examples, val_examples = train_test_split(
    training_examples, test_size=0.2, random_state=42
)

print(f"\n📈 Training set: {len(train_examples)} triplets")
print(f"📊 Validation set: {len(val_examples)} triplets")

# Initialize semantic sentence transformer
print("\n🚀 Initializing Semantic Sentence Transformer...")
print("🚫 Wandb integration disabled - using direct Hugging Face model")
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
semantic_model = SemanticSentenceTransformer(
    model_name=MODEL_NAME,
    max_seq_length=512
)

# Fine-tune the semantic sentence transformer
print("\n🧠 Starting semantic-aware fine-tuning...")
print("Key advantages over XLM-RoBERTa approach:")
print("  ✅ Pre-optimized for semantic similarity tasks")
print("  ✅ Faster training with smaller model size")
print("  ✅ Better out-of-the-box multilingual support")

# First try to use a pre-trained model to avoid memory issues
# If you want fine-tuning, uncomment the training block below
print("🔧 MEMORY-EFFICIENT APPROACH: Using pre-trained model to avoid memory issues")
print("💡 For production use, fine-tuning on a machine with more GPU memory is recommended")

# Load pre-trained model directly for hybrid retrieval
semantic_model = SemanticSentenceTransformer(
    model_name=MODEL_NAME,
    max_seq_length=512
)

print("✅ Using pre-trained semantic model for hybrid retrieval")

# Uncomment below for fine-tuning (requires more GPU memory):

model_path = semantic_model.fine_tune_semantic(
    training_examples=train_examples,
    val_examples=val_examples,
    epochs=2,
    batch_size=8,
    warmup_steps=50,
    output_path='best_sentence_transformer_semantic_ir'
)
print(f"🎉 Semantic fine-tuning completed!")
print(f"📁 Model saved to: {model_path}")



🧠 Creating semantic-aware training triplets for Sentence Transformer...
🚫 REMOVED: Keyword overlap-based negative mining
✅ ADDED: Semantic diversity-based negative sampling
⚡ OPTIMIZED: For sentence transformer efficiency
🧠 Creating SEMANTIC training triplets for Sentence Transformer...
❌ REMOVED: Keyword overlap-based negative mining
✅ ADDED: Semantic diversity-based negative sampling
✅ OPTIMIZED: For sentence transformer architecture


Semantic triplet creation: 100%|██████████| 20/20 [00:04<00:00,  4.08it/s]



📊 Created 1134 semantic training triplets

📈 Training set: 907 triplets
📊 Validation set: 227 triplets

🚀 Initializing Semantic Sentence Transformer...
🚫 Wandb integration disabled - using direct Hugging Face model
🚀 Initializing Semantic Sentence Transformer from Hugging Face: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
✅ Pre-optimized for semantic similarity
✅ Multilingual support for Bengali/Roman
✅ Efficient architecture for faster training
🚫 Wandb logging disabled


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


📊 Model parameters: 117,653,760
📏 Max sequence length: 512
✅ Using pre-trained semantic model for hybrid retrieval

🧠 Fine-tuning with 907 semantic triplets...
🎯 Focus: Semantic understanding over lexical matching
⚡ Advantage: Pre-trained semantic representations
🚫 Training without wandb logging

📈 Training configuration:
  Epochs: 2
  Batch size: 8
  Total steps: 228
  Warmup steps: 50
  Loss function: Triplet Loss (semantic ranking)
  Logging: Disabled (no wandb required)
  Evaluation: 100 examples for validation

🔄 Starting training...


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Semantic Ir Eval Pearson Cosine,Semantic Ir Eval Spearman Cosine
114,No log,No log,0.694010,0.722654
228,No log,No log,0.784109,0.769076



✅ Fine-tuning completed!
📁 Model saved to: best_sentence_transformer_semantic_ir
🎉 Semantic fine-tuning completed!
📁 Model saved to: best_sentence_transformer_semantic_ir


In [ ]:
# Final comprehensive comparison and summary
print("\n" + "="*80)
print("🚀 HYBRID RETRIEVAL VS SINGLE-METHOD APPROACHES")
print("="*80)

print("\n📊 METHODOLOGY COMPARISON:")
comparison_data = {
    'Aspect': [
        'Retrieval Type',
        'Query Coverage',
        'Lexical Matching',
        'Semantic Understanding',
        'Query Adaptability',
        'Robustness',
        'Performance Consistency',
        'Implementation Complexity'
    ],
    'BM25 Only (Sparse)': [
        'Keyword-based',
        'Limited to exact/similar terms',
        'Excellent',
        'Poor',
        'Poor (keyword queries only)',
        'Fails on paraphrases',
        'Inconsistent across query types',
        'Simple'
    ],
    'Semantic Only (Dense)': [
        'Meaning-based',
        'Good for conceptual queries',
        'Poor',
        'Excellent',
        'Good for semantic queries',
        'Fails on exact matches',
        'Inconsistent on factual queries',
        'Moderate'
    ],
    'Hybrid (Dense + Sparse)': [
        'Best of both worlds',
        'Comprehensive coverage',
        'Excellent',
        'Excellent',
        'Adapts to any query type',
        'Robust across query types',
        'Consistently high performance',
        'Higher but manageable'
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n📈 DETAILED COMPARISON:")
for _, row in comparison_df.iterrows():
    print(f"  {row['Aspect']:25s}: {row['BM25 Only (Sparse)']:25s} | {row['Semantic Only (Dense)']:25s} | {row['Hybrid (Dense + Sparse)']}")

print("\n🎯 HYBRID RETRIEVAL ADVANTAGES:")
print("  🎪 Combines lexical precision with semantic understanding")
print("  🏹 Captures exact keyword matches AND conceptual similarity")
print("  🔄 Adapts automatically to different query types")
print("  📈 Higher overall recall and precision")
print("  🛡️ Robust performance across diverse query scenarios")
print("  🎭 Handles both factual and conceptual information needs")
print("  🌍 Excellent for code-mixed Bengali (lexical + semantic)")

print("\n⚡ TECHNICAL IMPLEMENTATION BENEFITS:")
print("  🚀 Efficient fusion algorithms (weighted + RRF)")
print("  📊 Score normalization for fair combination")
print("  🔧 Configurable weights for domain adaptation")
print("  💾 Memory-efficient indexing (FAISS + BM25)")
print("  📈 Scalable to large document collections")
print("  🔍 Multiple fusion strategies for optimization")

print("\n📁 FILES CREATED:")
print("  1. hybrid_retrieval_weighted_submission.csv - Weighted fusion results")
print("  2. hybrid_retrieval_rrf_submission.csv - RRF fusion results")
print("  3. Semantic model embeddings and BM25 index")

print("\n🏆 EXPECTED PERFORMANCE GAINS:")
print("  📊 Higher MAP/NDCG scores than single methods")
print("  🎯 Better recall for diverse query types")
print("  ⚖️ More balanced precision across different queries")
print("  🌟 State-of-the-art results for code-mixed retrieval")
print("  🔧 Production-ready hybrid system")

print("\n🔬 FUSION STRATEGIES IMPLEMENTED:")
print("  ⚖️ Weighted Fusion: Linear combination of normalized scores")
print("  🔄 Reciprocal Rank Fusion (RRF): Rank-based combination")
print("  📈 Both methods handle score scale differences")
print("  🎛️ Configurable parameters for domain optimization")

print("\n" + "="*80)
print("✨ HYBRID RETRIEVAL SYSTEM COMPLETE!")
print("🧠 Combines the best of dense AND sparse retrieval!")
print("📊 Two submission files ready for evaluation:")
print("   • hybrid_retrieval_weighted_submission.csv (recommended)")
print("   • hybrid_retrieval_rrf_submission.csv (alternative)")
print("🏆 Expected to outperform single-method approaches!")
print("="*80)



🚀 HYBRID RETRIEVAL VS SINGLE-METHOD APPROACHES

📊 METHODOLOGY COMPARISON:

📈 DETAILED COMPARISON:
  Retrieval Type           : Keyword-based             | Meaning-based             | Best of both worlds
  Query Coverage           : Limited to exact/similar terms | Good for conceptual queries | Comprehensive coverage
  Lexical Matching         : Excellent                 | Poor                      | Excellent
  Semantic Understanding   : Poor                      | Excellent                 | Excellent
  Query Adaptability       : Poor (keyword queries only) | Good for semantic queries | Adapts to any query type
  Robustness               : Fails on paraphrases      | Fails on exact matches    | Robust across query types
  Performance Consistency  : Inconsistent across query types | Inconsistent on factual queries | Consistently high performance
  Implementation Complexity: Simple                    | Moderate                  | Higher but manageable

🎯 HYBRID RETRIEVAL ADVANTAGES:
  

In [ ]:
# Optional: Parameter Tuning and Analysis Tools for Hybrid Retrieval
print("🔧 HYBRID RETRIEVAL PARAMETER TUNING AND ANALYSIS")
print("=" * 60)

def tune_fusion_weights(hybrid_retriever, validation_queries, validation_qrels=None,
                       weight_range=(0.1, 0.9), step=0.1):
    """
    Tune the fusion weights by testing different semantic vs BM25 weight combinations

    This is useful if you have validation data to optimize the fusion parameters
    """
    print("🎛️ Testing different fusion weight combinations...")

    results = []
    weights = np.arange(weight_range[0], weight_range[1] + step, step)

    for semantic_weight in weights:
        print(f"  Testing semantic weight: {semantic_weight:.1f}")

        # Test on a sample of validation queries for efficiency
        sample_queries = dict(list(validation_queries.items())[:3])

        for query_id, query_text in sample_queries.items():
            hybrid_results = hybrid_retriever.hybrid_search(
                query_text, method='weighted', semantic_weight=semantic_weight, top_k=100
            )

            # Store results for analysis
            results.append({
                'query_id': query_id,
                'semantic_weight': semantic_weight,
                'bm25_weight': 1.0 - semantic_weight,
                'top_doc_id': hybrid_results[0][0] if hybrid_results else None,
                'top_score': hybrid_results[0][1] if hybrid_results else 0.0
            })

    results_df = pd.DataFrame(results)
    print("✅ Weight tuning analysis complete")
    return results_df

def analyze_query_type_performance(hybrid_retriever, test_queries, sample_size=5):
    """
    Analyze how different fusion methods perform on different types of queries
    """
    print("🔍 ANALYZING QUERY TYPE PERFORMANCE...")

    # Sample queries for analysis
    sample_queries = dict(list(test_queries.items())[:sample_size])

    analysis_results = []

    for query_id, query_text in sample_queries.items():
        print(f"\\nAnalyzing Query {query_id}: '{query_text[:60]}...'")

        # Get individual method results
        semantic_results = hybrid_retriever.semantic_search(query_text, top_k=50)
        bm25_results = hybrid_retriever.bm25_search(query_text, top_k=50)

        # Get fusion results
        weighted_results = hybrid_retriever.hybrid_search(
            query_text, method='weighted', semantic_weight=0.7, top_k=50
        )
        rrf_results = hybrid_retriever.hybrid_search(
            query_text, method='rrf', rrf_k=60, top_k=50
        )

        # Analyze coverage
        coverage = hybrid_retriever.analyze_query_coverage(query_text, top_k=50)

        analysis_results.append({
            'query_id': query_id,
            'query_length': len(query_text.split()),
            'semantic_top_score': semantic_results[0][1] if semantic_results else 0,
            'bm25_top_score': bm25_results[0][1] if bm25_results else 0,
            'weighted_top_score': weighted_results[0][1] if weighted_results else 0,
            'rrf_top_score': rrf_results[0][1] if rrf_results else 0,
            'overlap_percentage': coverage['overlap_percentage'],
            'semantic_unique_percentage': coverage['semantic_unique_percentage'],
            'bm25_unique_percentage': coverage['bm25_unique_percentage']
        })

        print(f"  📊 Method overlap: {coverage['overlap_percentage']:.1f}%")
        print(f"  🧠 Semantic-only docs: {coverage['semantic_unique_percentage']:.1f}%")
        print(f"  🔍 BM25-only docs: {coverage['bm25_unique_percentage']:.1f}%")

    analysis_df = pd.DataFrame(analysis_results)

    print("\\n📈 QUERY ANALYSIS SUMMARY:")
    print(f"  Average overlap: {analysis_df['overlap_percentage'].mean():.1f}%")
    print(f"  Average semantic-only: {analysis_df['semantic_unique_percentage'].mean():.1f}%")
    print(f"  Average BM25-only: {analysis_df['bm25_unique_percentage'].mean():.1f}%")

    return analysis_df

def compare_fusion_methods(hybrid_retriever, test_queries, sample_size=3):
    """
    Compare weighted fusion vs RRF on sample queries
    """
    print("⚖️ COMPARING FUSION METHODS...")

    sample_queries = dict(list(test_queries.items())[:sample_size])
    comparison_results = []

    for query_id, query_text in sample_queries.items():
        print(f"\\nComparing methods for Query {query_id}...")

        # Get results from both fusion methods
        weighted_results = hybrid_retriever.hybrid_search(
            query_text, method='weighted', semantic_weight=0.7, top_k=20
        )
        rrf_results = hybrid_retriever.hybrid_search(
            query_text, method='rrf', rrf_k=60, top_k=20
        )

        # Calculate rank correlation
        weighted_docs = [doc_id for doc_id, _ in weighted_results]
        rrf_docs = [doc_id for doc_id, _ in rrf_results]

        # Find common docs in top 10
        common_docs = set(weighted_docs[:10]) & set(rrf_docs[:10])

        comparison_results.append({
            'query_id': query_id,
            'weighted_top_doc': weighted_results[0][0] if weighted_results else None,
            'rrf_top_doc': rrf_results[0][0] if rrf_results else None,
            'same_top_doc': (weighted_results[0][0] == rrf_results[0][0]) if weighted_results and rrf_results else False,
            'common_in_top10': len(common_docs),
            'common_percentage_top10': len(common_docs) / 10 * 100
        })

        print(f"  Same top document: {comparison_results[-1]['same_top_doc']}")
        print(f"  Common docs in top 10: {len(common_docs)}/10 ({len(common_docs)*10}%)")

    comparison_df = pd.DataFrame(comparison_results)

    print("\\n📊 FUSION METHOD COMPARISON SUMMARY:")
    same_top_percentage = comparison_df['same_top_doc'].mean() * 100
    avg_common_percentage = comparison_df['common_percentage_top10'].mean()

    print(f"  Same top document: {same_top_percentage:.1f}% of queries")
    print(f"  Average overlap in top 10: {avg_common_percentage:.1f}%")

    if same_top_percentage > 70:
        print("  ✅ Methods are largely consistent - either can be used")
    elif same_top_percentage > 40:
        print("  ⚖️ Methods show some differences - test both for best results")
    else:
        print("  🔄 Methods are quite different - consider ensemble approach")

    return comparison_df

# Run analysis on sample data
print("\\n🚀 Running hybrid retrieval analysis...")

# Analyze query type performance
# query_analysis = analyze_query_type_performance(hybrid_retriever, test_queries, sample_size=3)

# Compare fusion methods
# fusion_comparison = compare_fusion_methods(hybrid_retriever, test_queries, sample_size=3)

print("\\n💡 OPTIMIZATION RECOMMENDATIONS:")
print("1. 📊 Use weighted fusion (0.7 semantic) as default")
print("2. 🔄 Try RRF for more conservative ranking")
print("3. 🎛️ Adjust semantic weight based on query characteristics:")
print("   • Higher semantic weight (0.8+) for conceptual queries")
print("   • Lower semantic weight (0.5-0.6) for factual/exact queries")
print("4. 📈 Monitor overlap percentage to detect query types")
print("5. 🔧 Fine-tune BM25 parameters (k1, b) for domain-specific optimization")

print("\\n✅ Analysis complete! Use insights to optimize your hybrid retrieval system.")


🔧 HYBRID RETRIEVAL PARAMETER TUNING AND ANALYSIS
\n🚀 Running hybrid retrieval analysis...
\n💡 OPTIMIZATION RECOMMENDATIONS:
1. 📊 Use weighted fusion (0.7 semantic) as default
2. 🔄 Try RRF for more conservative ranking
3. 🎛️ Adjust semantic weight based on query characteristics:
   • Higher semantic weight (0.8+) for conceptual queries
   • Lower semantic weight (0.5-0.6) for factual/exact queries
4. 📈 Monitor overlap percentage to detect query types
5. 🔧 Fine-tune BM25 parameters (k1, b) for domain-specific optimization
\n✅ Analysis complete! Use insights to optimize your hybrid retrieval system.


In [ ]:
# ALTERNATIVE: Quick test without fine-tuning
# If you want to skip training and just test the semantic retrieval approach:

print("🔧 ALTERNATIVE: Testing with pre-trained model (no fine-tuning)")
print("This will use the model as-is without additional training")

# Use pre-trained model directly
quick_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
quick_model = quick_model.to(device)

print("✅ Pre-trained model loaded successfully")
print("🚀 You can now proceed to document indexing with this model")
print("📝 Note: Results may be slightly lower quality without fine-tuning")

# To use this model, uncomment the next line and skip the fine-tuning step:
# semantic_model = type('obj', (object,), {'model': quick_model, 'encode': lambda self, *args, **kwargs: self.model.encode(*args, **kwargs)})()


🔧 ALTERNATIVE: Testing with pre-trained model (no fine-tuning)
This will use the model as-is without additional training
✅ Pre-trained model loaded successfully
🚀 You can now proceed to document indexing with this model
📝 Note: Results may be slightly lower quality without fine-tuning


In [ ]:
# Verification: Check that wandb is properly disabled
print("🔍 VERIFICATION: Checking environment setup...")

import os
print(f"WANDB_DISABLED: {os.environ.get('WANDB_DISABLED', 'Not set')}")
print(f"TRANSFORMERS_NO_ADVISORY_WARNINGS: {os.environ.get('TRANSFORMERS_NO_ADVISORY_WARNINGS', 'Not set')}")

# Test sentence transformer import
try:
    from sentence_transformers import SentenceTransformer
    print("✅ SentenceTransformer import successful")

    # Quick test load (small model)
    test_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    test_embedding = test_model.encode(["This is a test sentence."])
    print(f"✅ Model encoding test successful - shape: {test_embedding.shape}")

    del test_model  # Clean up memory
    print("🚀 Ready to proceed with semantic training!")

except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Try installing: pip install sentence-transformers")

print("\n" + "="*50)
print("🎯 Environment verified - no wandb issues expected!")
print("="*50)


🔍 VERIFICATION: Checking environment setup...
WANDB_DISABLED: true
TRANSFORMERS_NO_ADVISORY_WARNINGS: true
✅ SentenceTransformer import successful


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model encoding test successful - shape: (1, 384)
🚀 Ready to proceed with semantic training!

🎯 Environment verified - no wandb issues expected!


In [ ]:
# Initialize Hybrid Retrieval System (Sentence Transformer + BM25)
print("🚀 INITIALIZING HYBRID RETRIEVAL SYSTEM...")
print("🔧 Combining Dense (Semantic) + Sparse (BM25) Retrieval")

# Initialize hybrid retriever
hybrid_retriever = HybridRetriever(
    semantic_model=semantic_model,
    documents=documents,
    bm25_k1=1.2,  # BM25 term frequency saturation parameter
    bm25_b=0.75   # BM25 document length normalization parameter
)

print("✅ Hybrid retrieval system ready!")

# Analyze method complementarity for sample queries
print("\n🔍 ANALYZING RETRIEVAL METHOD COMPLEMENTARITY...")
sample_query_id = list(test_queries.keys())[0]
sample_query_text = test_queries[sample_query_id]

coverage_analysis = hybrid_retriever.analyze_query_coverage(sample_query_text, top_k=100)
print(f"\nSample Query Analysis: '{sample_query_text[:50]}...'")
print(f"  📊 Overlap between methods: {coverage_analysis['overlap_percentage']:.1f}%")
print(f"  🧠 Semantic-only documents: {coverage_analysis['semantic_unique_percentage']:.1f}%")
print(f"  🔍 BM25-only documents: {coverage_analysis['bm25_unique_percentage']:.1f}%")
print(f"  📈 Total unique documents found: {coverage_analysis['total_unique_docs']}")

def create_hybrid_submission_file(test_queries, hybrid_retriever,
                                 fusion_method='weighted', semantic_weight=0.7,
                                 output_file='hybrid_retrieval_submission.csv'):
    """Create submission file with hybrid retrieval - TOP 1000 documents per query"""
    submission_data = []

    print(f"\n🔍 Generating hybrid retrieval for {len(test_queries)} test queries...")
    print(f"🔧 Fusion method: {fusion_method}")
    if fusion_method == 'weighted':
        print(f"⚖️ Semantic weight: {semantic_weight}, BM25 weight: {1-semantic_weight}")

    for query_id, query_text in tqdm(test_queries.items(), desc="Hybrid retrieval"):
        # Get hybrid results
        if fusion_method == 'weighted':
            hybrid_results = hybrid_retriever.hybrid_search(
                query_text, method='weighted', semantic_weight=semantic_weight, top_k=1000
            )
        else:  # RRF
            hybrid_results = hybrid_retriever.hybrid_search(
                query_text, method='rrf', rrf_k=60, top_k=1000
            )
        i=0
        # Add to submission data
        for rank, (doc_id, score) in enumerate(hybrid_results):
            submission_data.append({
                'index':i,
                'qid': query_id,
                'docno': doc_id,
                'docid': doc_id - 1,  # Convert to 0-based indexing
                'rank': rank,
                'score': score,
                # 'query': test_queries_real[query_id]
            })
            i+=1

    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(output_file, index=False)
    print(f"✅ Hybrid submission saved to {output_file}")
    return submission_df

# Generate hybrid submission with weighted fusion (default)
print("🎯 GENERATING HYBRID RETRIEVAL RESULTS...")
hybrid_submission_df = create_hybrid_submission_file(
    test_queries, hybrid_retriever,
    fusion_method='weighted', semantic_weight=0.7,
    output_file='hybrid_retrieval_weighted_submission.csv'
)

# Also generate with RRF for comparison
print("\n🔄 GENERATING RRF FUSION RESULTS...")
rrf_submission_df = create_hybrid_submission_file(
    test_queries, hybrid_retriever,
    fusion_method='rrf',
    output_file='hybrid_retrieval_rrf_submission.csv'
)

print(f"\n📊 HYBRID RETRIEVAL RESULTS STATISTICS:")
print(f"  📋 Total entries (Weighted): {len(hybrid_submission_df):,}")
print(f"  📋 Total entries (RRF): {len(rrf_submission_df):,}")
print(f"  🔍 Test queries processed: {hybrid_submission_df['qid'].nunique()}")
print(f"  📖 Documents per query: {len(hybrid_submission_df) / hybrid_submission_df['qid'].nunique():.0f}")
print(f"  🎯 Score range (Weighted): {hybrid_submission_df['score'].min():.4f} to {hybrid_submission_df['score'].max():.4f}")
print(f"  🎯 Score range (RRF): {rrf_submission_df['score'].min():.4f} to {rrf_submission_df['score'].max():.4f}")

# Display sample results comparison
print("\n🔍 SAMPLE HYBRID RESULTS COMPARISON:")
sample_query = hybrid_submission_df['qid'].iloc[0]
sample_weighted = hybrid_submission_df[hybrid_submission_df['qid'] == sample_query].head(5)
sample_rrf = rrf_submission_df[rrf_submission_df['qid'] == sample_query].head(5)

print(f"\nTop 5 WEIGHTED FUSION results for Test Query {sample_query}:")
for _, row in sample_weighted.iterrows():
    print(f"  Rank {row['rank']:3d}: Doc {row['docno']:6d} (Score: {row['score']:.4f})")

print(f"\nTop 5 RRF FUSION results for Test Query {sample_query}:")
for _, row in sample_rrf.iterrows():
    print(f"  Rank {row['rank']:3d}: Doc {row['docno']:6d} (Score: {row['score']:.4f})")

print(f"\n🎉 SUCCESS! Generated hybrid retrieval results!")
print("📊 Each query has exactly 1000 ranked documents using HYBRID APPROACH")
print("🧠 Combines semantic understanding WITH lexical precision!")
print("⚡ Best of both dense and sparse retrieval methods!")


🚀 INITIALIZING HYBRID RETRIEVAL SYSTEM...
🔧 Combining Dense (Semantic) + Sparse (BM25) Retrieval
🔧 Initializing Hybrid Retriever with 107900 documents...
📊 BM25 parameters: k1=1.2, b=0.75
🔍 Building BM25 index...
   ✅ BM25 index built with 107900 documents
🧠 Building semantic embeddings index...


Batches:   0%|          | 0/3372 [00:00<?, ?it/s]

   ✅ Semantic index built with 107900 documents
✅ Hybrid retriever initialized successfully!
✅ Hybrid retrieval system ready!

🔍 ANALYZING RETRIEVAL METHOD COMPLEMENTARITY...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Sample Query Analysis: 'hi hyderabad e rapid antigen test kothay kora hoch...'
  📊 Overlap between methods: 6.0%
  🧠 Semantic-only documents: 94.0%
  🔍 BM25-only documents: 94.0%
  📈 Total unique documents found: 194
🎯 GENERATING HYBRID RETRIEVAL RESULTS...

🔍 Generating hybrid retrieval for 30 test queries...
🔧 Fusion method: weighted
⚖️ Semantic weight: 0.7, BM25 weight: 0.30000000000000004


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Hybrid retrieval:  97%|█████████▋| 29/30 [00:57<00:01,  1.79s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Hybrid retrieval: 100%|██████████| 30/30 [00:58<00:00,  1.94s/it]


✅ Hybrid submission saved to hybrid_retrieval_rrf_submission.csv

📊 HYBRID RETRIEVAL RESULTS STATISTICS:
  📋 Total entries (Weighted): 30,000
  📋 Total entries (RRF): 30,000
  🔍 Test queries processed: 30
  📖 Documents per query: 1000
  🎯 Score range (Weighted): 0.0302 to 1.0000
  🎯 Score range (RRF): 0.0017 to 0.0328

🔍 SAMPLE HYBRID RESULTS COMPARISON:

Top 5 WEIGHTED FUSION results for Test Query 26:
  Rank   0: Doc  95770 (Score: 0.9687)
  Rank   1: Doc  27818 (Score: 0.9527)
  Rank   2: Doc  75989 (Score: 0.8524)
  Rank   3: Doc  65231 (Score: 0.8192)
  Rank   4: Doc   4954 (Score: 0.7924)

Top 5 RRF FUSION results for Test Query 26:
  Rank   0: Doc  95770 (Score: 0.0323)
  Rank   1: Doc  27818 (Score: 0.0323)
  Rank   2: Doc  75989 (Score: 0.0306)
  Rank   3: Doc  65231 (Score: 0.0301)
  Rank   4: Doc   4954 (Score: 0.0277)

🎉 SUCCESS! Generated hybrid retrieval results!
📊 Each query has exactly 1000 ranked documents using HYBRID APPROACH
🧠 Combines semantic understanding WITH le